In [1]:
%matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm

from scipy.signal import argrelextrema
import scipy.optimize as opt

import numpy as np

Using matplotlib backend: Qt5Agg


In [2]:
# read data
stars = np.loadtxt('isochrone/stars_std.lst')

iso1 = np.loadtxt('isochrone/iso_4p2Gyr_z0_d1Kpc_r0p03.dat')
iso2 = np.loadtxt('isochrone/iso_4p2Gyr_z0_d830pc_r0p03.dat')
iso3 = np.loadtxt('isochrone/iso_4p4Gyr_z0_d830pc_r0p03.dat')
iso4 = np.loadtxt('isochrone/iso_a4000Myr_z0p01_d850pc_r0p04.dat')
iso5 = np.loadtxt('isochrone/iso_a5000Myr_z0p02_d1Mpc_r0')

In [3]:
# plot CMD

# with errorbars
#plt.errorbar(stars[:,2], stars[:,0], yerr=stars[:,1],xerr=stars[:,3],linewidth=0,elinewidth=1,label='Observations',color='k') #x-axis: b-y; y-axis: V

# without errorbars
plt.plot(stars[:,2], stars[:,0],'.',label='Observations',color='k',markersize=2)

# plot isochrones on top of CMD
#plt.plot(iso1[:,1], iso1[:,0], '-', label='iso1')
plt.plot(iso2[:,1], iso2[:,0], 'r-', label='Best isochrone fit')
#plt.plot(iso3[:,1], iso3[:,0], '-', label='iso3')
#plt.plot(10**iso4[:,3], iso4[:,2], '-', label='iso4')
#plt.plot(10**iso5[:,3], iso5[:,2], '-', label='iso5')

# Set plot parameters and labels
plt.ylim([20, 9])
#plt.xlim([10000, 3000])
plt.xlabel('$b-y$')
plt.ylabel('$V$')
plt.legend()

In [4]:
# fem el fit en questió. aixo no ho he borrat perque va be per 
# veure com es comporten les funcions fora de la zona MS.
# així, com més lineal és menys modifica sense sentit els resultats
# de les estrelles fora de la MS.
def curvefitter(x,p0,plow,phigh,pchange,prate):
    # params has dimension 5 (from 0 to 4)
    #y = p0+p1*x+p2*x**2+p3*x**3+p_1/x
    # try with variable p1
    p1 = (phigh-plow)/(1+np.exp(prate*(x-pchange)))+plow
    y = p0+p1*(x-pchange)
    return y

In [35]:
def Vby2LT(stars,iso,plot):
    # 1- get indMax
    maxInd = argrelextrema(iso[:,3], np.less)[0][0]
    by = iso[:maxInd,1]
    T  = iso[:maxInd,3]
    V  = iso[:maxInd,0]
    L  = iso[:maxInd,2]
    # 2- fit the curves
    # 2.1- fit T(by)
    plow = (T[10]-T[0])/(by[10]-by[0])
    phigh = (T[-1]-T[-10])/(by[-1]-by[-10])
    p0_0 = T[0]-plow*by[0]
    p0 = [p0_0,plow,phigh,.6,10]
    poptT, pcovT = opt.curve_fit(curvefitter,by,T,p0)
    # 2.1- fit T(by)
    plow = (L[10]-L[0])/(V[10]-V[0])
    phigh = (L[-90]-L[-100])/(V[-90]-V[-100])
    p0_0 = -4.05-plow*16
    p0 = [p0_0,plow*1.1,phigh,16,1]
    poptL, pcovL = opt.curve_fit(curvefitter,V,L,p0)
    print 'MSE (mean,total)'
    Tfit = curvefitter(by,*poptT)
    Lfit = curvefitter(V,*poptL)
    MSET = (T-Tfit)**2
    MSEL = (L-Lfit)**2
    print 'L',np.mean(MSEL),sum(MSEL),len(MSEL)
    print 'Teff',np.mean(MSET),sum(MSET),len(MSET)
    # 3- apply the curve with the fitted parameters to V and by to 
    # get L and T
    Lstars = curvefitter(stars[:,0],*poptL)
    Tstars = curvefitter(stars[:,2],*poptT)
    print '\nParameters and coefs\nL'
    print poptL
    print np.sqrt(np.diag(pcovL))
    print 'Teff'
    print poptT
    print np.sqrt(np.diag(pcovT))
    if plot:
        plt.figure(figsize=(10,6))
        plt.subplot(121)
        plt.plot(by,T,'r',label='isochrone data')
        plt.plot(by,Tfit,'k-.',label='fit')
        plt.xlabel(r'$b-y$')
        plt.ylabel(r'$T_{eff}\ (K)$')
        plt.legend()
        plt.subplot(122)
        plt.plot(V,L,'k',label='isochrone data',lw=1.5)
        plt.plot(V,Lfit,'r-.',label='fit',lw=1.5)
        plt.xlabel(r'$V$')
        plt.ylabel(r'$L\ (L_\odot)$')
        plt.legend()
        plt.tight_layout()
    return Tstars,Lstars

In [36]:
Tstars,Lstars = Vby2LT(stars,iso2,True)

MSE (mean,total)
L 2.284183627457139e-06 0.0014047729308861415 615
Teff 1.5426984548292978e-07 9.48759549720017e-05 615

Parameters and coefs
L
[-0.30166921 -0.26973369 -0.39581946 15.41291052  0.75885954]
[0.00422819 0.00039563 0.00107449 0.0126118  0.01189479]
Teff
[ 3.66641153 -0.27395584 -0.46822826  0.63550082 15.39384102]
[0.00020929 0.00052241 0.0003003  0.00055179 0.19704135]


In [39]:
plt.figure()
plt.semilogy(10**iso2[:,3],10**iso2[:,2],'-',label='Best isochrone fit',color='purple')
plt.scatter(10**Tstars,10**Lstars,c=10**Tstars,label='Observations',cmap=cm.RdYlBu, vmin=3000,
            vmax=9000,marker='.',s=1.2)#,linewidths=0.1,edgecolors='k')
plt.xlim([10000, 3000])
plt.xlabel('$T_{eff}$ (K)')
plt.ylabel('$L/L_\odot$')
plt.gca().set_facecolor('xkcd:almost black') # sensible options: charcoal, dark, almost black and midnight. Black is too much
#plt.grid(True,color='xkcd:light grey', linestyle='--', linewidth=1, alpha=0.6)
plt.legend()

## Generate the list of pst stars

In [4]:
pst_b = np.loadtxt('Lab1/science_m67_b.pst.1',dtype=np.float64)
pst_b = pst_b[pst_b[:,0].argsort()] # sort array according to first column
pst_y = np.loadtxt('Lab1/science_m67_y.pst.1',dtype=np.float64)
pst_y = pst_y[pst_y[:,0].argsort()] # sort array according to first column

In [5]:
print 'b filter\n'
for i,(num,x,y,m,ms) in enumerate(pst_b):
    print '%d & %.3f & %.3f & %.3f & %.7g \\\\' %(num,x,y,m,ms)
print '\ny filter\n'
for i,(num,x,y,m,ms) in enumerate(pst_y):
    print '%d & %.3f & %.3f & %.3f & %.7g \\\\' %(num,x,y,m,ms)

b filter

12 & 1225.507 & 52.826 & -10.981 & -6.149918 \\
24 & 1829.076 & 114.152 & -11.659 & -0.9860825 \\
31 & 1436.586 & 157.905 & -10.313 & -9.385581 \\
34 & 846.279 & 174.193 & -10.172 & -8.53876 \\
35 & 798.813 & 179.623 & -10.855 & -6.703156 \\
36 & 570.607 & 180.730 & -11.212 & -5.184465 \\
65 & 504.365 & 314.500 & -10.385 & -8.76615 \\
66 & 812.461 & 318.594 & -10.494 & -7.911538 \\
86 & 176.649 & 478.417 & -10.247 & -9.382069 \\
109 & 705.694 & 616.636 & -10.707 & -5.814357 \\
175 & 1945.945 & 876.506 & -11.745 & 1.119405 \\
185 & 1272.660 & 931.650 & -10.164 & -8.317695 \\
203 & 866.194 & 1006.945 & -11.080 & -4.050434 \\
222 & 1540.214 & 1097.801 & -11.020 & -3.684947 \\
226 & 358.469 & 1101.716 & -11.399 & -3.561035 \\
227 & 1580.952 & 1112.471 & -10.454 & -5.203313 \\
235 & 213.076 & 1145.705 & -11.611 & -1.427757 \\
241 & 1377.805 & 1173.293 & -10.654 & -6.27722 \\
253 & 819.568 & 1218.820 & -10.168 & -8.262375 \\
280 & 1653.738 & 1312.492 & -10.214 & -3.942057 \\
309 & 